import libraries

In [4]:
pip install opencv-python

Let's return video from the first webcam on your computer.

In [ ]:
importing the neccessary libraries

In [ ]:
import cv2
import time
import datetime
import smtplib



- Let's return video from the first webcam on your computer.

In [ ]:
cap = cv2.VideoCapture(0)

- OpenCV already contains many pre-trained classifiers for face, eyes, smile etc. Those XML files are stored in opencv/data/haarcascades/ folder
- so lets load in the face and body pre-trained classifiers

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
body_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_fullbody.xml")


- Add your email account credentials

In [ ]:
my_email = "youremail@gmail.com"
password = "password"

In [ ]:
next, we need to declare all our variables

In [ ]:
detection = False
detection_stopped_time = None
timer_started = False
SECONDS_TO_RECORD_AFTER_DETECTION = 5


In [ ]:
next, we get the frame size and then the frame character code which is like the format the video will be saved as

In [ ]:
frame_size = (int(cap.get(3)), int(cap.get(4)))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")



- lets convert our frame into a greyscale image. all these classifiers requires a greyscale image to do the classification
- it will tell us the presence and location of any image in the greyscale image
- the gray variable, returns a list of positions of faces that exist such as the x,y, width and height
- we then detect the faces and bodies in the gray scale

In [ ]:
while True:
    _, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.05, 4)
    bodies = face_cascade.detectMultiScale(gray, 1.05, 4) 
   

- have we seen or detected any face or bodies? if yes, then we ask, are we currently recording?

- have we seen or detected any face or bodies? if yes, then we ask, are we currently recording

In [ ]:


    if len(faces) + len(bodies) > 0:
        if detection:
            timer_started = False
           

- but if we were not recording before, then we need to since we have detected something.
- get the current time
- use that as our file name
- start the video output string

In [ ]:
 
        else:
            detection = True
            current_time = datetime.datetime.now().strftime("%d-%m-%y-%H-%M-%S")
            out = cv2.VideoWriter(f"{current_time}.mp4", fourcc, 20, frame_size)
            print("started recording")
            with smtplib.SMTP_SSL("smtp.gmail.com", 465) as connection:
                connection.ehlo()
                connection.login(user=my_email, password=password)
                connection.sendmail(from_addr=my_email, to_addrs=my_email,
                                    msg=f"Subject: Movement tracker\n\n"
                                        f"your movement tracker started at: {current_time}")
    

- continuing, we go to the elif, if we have not detected anything and we are recording, then its time to start the timer and see if any faces or bodies come into the camera withing that time or duration we already set
- if the timer has started we check to see if the timer has expired, 
- if it has, then we stop the recording
- if it hasnt we just then continue

if the timer has not started we start it and keep track of when the timer was started

In [ ]:
elif detection:
        if timer_started:
            if time.time() - detection_stopped_time >= SECONDS_TO_RECORD_AFTER_DETECTION:
                detection = False
                timer_started = False
                out.release()
                print("Stop recording!")
                with smtplib.SMTP_SSL("smtp.gmail.com", 465) as connection:
                    connection.ehlo()
                    connection.login(user=my_email, password=password)
                    connection.sendmail(from_addr=my_email, to_addrs=my_email,
                                        msg=f"Subject: Movement tracker\n\n"
                                            f"your movement tracker stopped at: {current_time}")
        else:
            timer_started = True
            detection_stopped_time = time.time()

    

In [ ]:
then we check if we are detecting something? if we are, then write
then we show the current frame
and press 'q' if you want to quit

In [ ]:
if detection:
        out.write(frame)
    
    cv2.imshow("camera", frame)
    if cv2.waitKey(1) == ord("q"):
        break


In [ ]:
finally, we release all our resources and destroy all open cv windows

In [ ]:
out.release()
cap.release()
cv2.destroyAllWindows()